In [ ]:
!pip install cython

In [ ]:
!pip install "pystan<2.18"

In [ ]:
!pip install fbprophet

In [ ]:
# library import
import pandas as pd
from matplotlib import pyplot
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import init_notebook_mode

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# data load
def load_data():
  df = pd.read_csv("data/아파트 실거래가격지수.csv")
  df.rename(columns={"Unnamed: 0" : "Column1"}, inplace=True)
  df = df.set_index("Column1").reset_index()

  return df

# prophet modeling
def m_prophet(target,period):
  df = load_data()
  df = df[["Column1",target]]
    
  df.columns = ['ds','y']

  m = Prophet(yearly_seasonality=10, weekly_seasonality=False, daily_seasonality=False)
  m.fit(df)

  future = m.make_future_dataframe(periods=period, freq="MS")
    
  forecast = m.predict(future)[['ds','yhat','yhat_upper','yhat_lower']]
  # m.plot(forecast)

  return forecast

In [ ]:
target_list = ['도심권[2017.11=100]', '동북권[2017.11=100]', '동남권[2017.11=100]',
       '부산[2017.11=100]', '대구[2017.11=100]', '인천[2017.11=100]',
       '광주[2017.11=100]', '대전[2017.11=100]', '울산[2017.11=100]',
       '세종[2017.11=100]', '경기[2017.11=100]', '강원[2017.11=100]',
       '충북[2017.11=100]', '충남[2017.11=100]', '서북권[2017.11=100]',
       '서남권[2017.11=100]', '전북[2017.11=100]', '전남[2017.11=100]',
       '경북[2017.11=100]', '경남[2017.11=100]', '제주[2017.11=100]',
       '광역시[2017.11=100]', '지방광역시[2017.11=100]', '지방도[2017.11=100]',
       '전국[2017.11=100]', '서울[2017.11=100]', '수도권[2017.11=100]',
       '지방[2017.11=100]']

In [ ]:
final = pd.DataFrame()
for target in target_list:
  if target == "광역시[2017.11=100]" or target == "광역시[2017.11=100]_yhat" or target == "광역시[2017.11=100]_yhat_upper" or target == "광역시[2017.11=100]_yhat_upper":
    pass

  else:
    
    dd = load_data()
    forecast = m_prophet(target,18) # 18개월 예측
    sub = pd.DataFrame()
    sub['date'] = pd.date_range(start='2006-01-01', end='2022-07-01', freq='MS')
    sub['city'] = target
    sub['yhat'] = list(forecast["yhat"])
    sub['yhat_upper'] = list(forecast["yhat_upper"])
    sub['yhat_lower'] = list(forecast["yhat_lower"])

    sub2 = pd.DataFrame()
    sub2['date'] = pd.date_range(start='2006-01-01', end='2021-01-01', freq='MS')
    sub2['real'] = list(dd[target])

    sub3 = pd.merge(sub2,sub, how='right')

    final = pd.concat([final,sub3])

final.set_index("date", inplace=True)
final.to_excel('data/prep_실거래지수.xlsx')

* 외부 데이터
  - 주가 데이터(FinanceDataReader API)
  - KOSIS 국가통계포털(https://kosis.kr/index/index.do)